In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import time


def custom_mre(y_true, y_pred):
    """numerator = tf.reduce_mean(tf.abs(y_pred - y_true), axis=None)
    denominator = tf.reduce_mean(tf.abs(y_true), axis=None)
    mre = 100.0 * numerator / denominator
    return mre"""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    relative_error = np.abs((y_true - y_pred) / y_true)
    
    mre = np.mean(relative_error) * 100.0
    
    return mre

In [2]:
# Cargar los datos desde el archivo CSV (asegúrate de ajustar el nombre del archivo)
data = pd.read_csv('W168H24.csv')
data_original = pd.read_csv('dataset.csv')

# Eliminar la primera columna (índice 0)
data = data.iloc[:, 1:]

# Separar las características (entradas) y las etiquetas (salidas)
X = data.iloc[:, :168].values  # Primeras 168 columnas son características de entrada
y = data.iloc[:, 168:].values  # 24 columnas restantes son etiquetas de salida

# Dividir los datos en conjuntos de entrenamiento (70%) y prueba (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Dividir el conjunto de entrenamiento en entrenamiento y validación (70% - 30%)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

y_train = y_train.astype('float32')
y_val = y_val.astype('float32')
y_test = y_test.astype('float32')


In [3]:
# Definir el modelo de la red neuronal
model = keras.Sequential()

# Agregar capa de entrada
model.add(keras.layers.Input(shape=(168,)))  # Ahora son 168 características después de eliminar la primera columna

# Agregar capas ocultas con las neuronas especificadas
neuronas_ocultas = [30, 80, 90, 60, 60, 100, 40, 80, 30, 80, 50, 100, 40, 80, 90, 40, 70, 70, 70, 60, 90, 70, 100, 100]

for neurons in neuronas_ocultas:
    model.add(keras.layers.Dense(neurons, activation='tanh'))

# Agregar capa de salida con 24 neuronas (salidas)
model.add(keras.layers.Dense(24, activation='tanh'))

# Definir el optimizador
optimizer = keras.optimizers.Adam(epsilon=1E-8, learning_rate=0.0005)

# Compilar el modelo con el optimizador personalizado
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mean_squared_error','mean_absolute_error'])

# Resumen del modelo
model.summary()

# Configurar Early Stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Métrica a monitorear (en este caso, la pérdida en el conjunto de validación)
    patience=10,  # Número de épocas sin mejora antes de detener el entrenamiento
    min_delta=1E-4,  # Valor mínimo de mejora para considerar como una mejora significativa
    restore_best_weights=True  # Restaurar los mejores pesos del modelo cuando se detiene
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                5070      
                                                                 
 dense_1 (Dense)             (None, 80)                2480      
                                                                 
 dense_2 (Dense)             (None, 90)                7290      
                                                                 
 dense_3 (Dense)             (None, 60)                5460      
                                                                 
 dense_4 (Dense)             (None, 60)                3660      
                                                                 
 dense_5 (Dense)             (None, 100)               6100      
                                                                 
 dense_6 (Dense)             (None, 40)                4

In [4]:
# Entrenar el modelo con Early Stopping
epochs = 300  
batch_size = 256 
# Comienza a medir el tiempo de entrenamiento
start_time = time.time()
#Entrenamiento del modelo
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=[early_stopping])
# Finaliza la medición del tiempo de entrenamiento
end_time = time.time()

#Guardar el modelo entrenado
model_path = "modelTrainedv3.h5"
model.save(model_path)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Desnormalización de los datos
min_value = data_original['consumption'].min()
max_value = data_original['consumption'].max()

#Desnormalización de los valores
y_pred_desnormalized = y_pred * (max_value - min_value) + min_value
y_test_desnormalized = y_test * (max_value - min_value) + min_value

# Calcular RMSE con datos desnormalizados
rmse = np.sqrt(mean_squared_error(y_test_desnormalized, y_pred_desnormalized))
print(f'RMSE en el conjunto de prueba: {rmse}')

# Calcular MAE con datos desnormalizados
mae = mean_absolute_error(y_test_desnormalized, y_pred_desnormalized)
print(f'MAE en el conjunto de prueba: {mae}')

# Calcular MRE con datos desnormalizados
mre = custom_mre(y_test_desnormalized, y_pred_desnormalized)
print(f'MRE en el conjunto de prueba: {mre}')

#Calcular MAPE con datos desnormalizados
print("Mean absolute percentage error (MAPE):%f" % mean_absolute_percentage_error(y_test_desnormalized, y_pred_desnormalized))

# Calcula la duración del entrenamiento en segundos
training_duration = end_time - start_time
# Imprime el tiempo de entrenamiento en segundos y en formato de horas, minutos y segundos
print(f'Tiempo de entrenamiento: {training_duration:.2f} segundos')
print(f'Tiempo de entrenamiento (HH:MM:SS): {int(training_duration // 3600)}:{int((training_duration % 3600) // 60)}:{int(training_duration % 60)}')

Epoch 1/300
953/953 [==============================] - 8s 6ms/step - loss: 0.0410 - mean_squared_error: 0.0044 - val_loss: 0.0349 - val_mean_squared_error: 0.0023
Epoch 2/300
953/953 [==============================] - 5s 5ms/step - loss: 0.0270 - mean_squared_error: 0.0017 - val_loss: 0.0296 - val_mean_squared_error: 0.0018
Epoch 3/300
953/953 [==============================] - 5s 5ms/step - loss: 0.0240 - mean_squared_error: 0.0013 - val_loss: 0.0228 - val_mean_squared_error: 0.0011
Epoch 4/300
953/953 [==============================] - 5s 6ms/step - loss: 0.0226 - mean_squared_error: 0.0011 - val_loss: 0.0265 - val_mean_squared_error: 0.0013
Epoch 5/300
953/953 [==============================] - 5s 6ms/step - loss: 0.0205 - mean_squared_error: 8.5249e-04 - val_loss: 0.0231 - val_mean_squared_error: 9.5011e-04
Epoch 6/300
953/953 [==============================] - 5s 6ms/step - loss: 0.0195 - mean_squared_error: 7.6319e-04 - val_loss: 0.0176 - val_mean_squared_error: 6.5208e-04
Epoch 

c:\Users\AdrianGilGamboa\Desktop\PycharmProjects\venv\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4667/4667 [==============================] - 5s 985us/step
RMSE en el conjunto de prueba: 471.21063232421875
MAE en el conjunto de prueba: 331.0008850097656
MRE en el conjunto de prueba: 1.1600966565310955
Mean absolute percentage error (MAPE):0.011601
Tiempo de entrenamiento: 350.64 segundos
Tiempo de entrenamiento (HH:MM:SS): 0:5:50
